In [1]:
from quad import Quadrotor3D
from tqdm import tqdm
import pdb
import numpy as np
#from visualization import initialize_drone_plotter, draw_drone_simulation, trajectory_tracking_results
import my_viz
from utils import skew_symmetric, quaternion_to_euler, unit_quat, v_dot_q
from time import sleep
import os
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import casadi as cs
from acados_template import AcadosOcp, AcadosOcpSolver, AcadosModel

In [2]:
'''
#os.environ["PYTHONBREAKPOINT"] = "0"


real_time_artists = None
    
if __name__ == "__main__":
    my_quad = quad.Quadrotor3D()



    simulation_steps = 20000
    simulation_step_length = 0.1
    x_history = list()
    np.random.seed(0)
    t = np.arange(0, simulation_steps*simulation_step_length, simulation_step_length)

    quad_current_state = my_quad.get_state(quaternion=True, stacked=True)
    quad_trajectory = np.zeros((13, simulation_steps))
    #print(my_quad.get_state(quaternion=True))
    #print(range(t))
    for current_idx in tqdm(np.arange(1,simulation_steps)):
        u = np.random.random((4,))
        u = np.array([1,1,1,1])*10
        #print(u)
        my_quad.update(u, simulation_step_length)

        #breakpoint()
        #x_pred = np.zeros((13,1))
        quad_current_state = my_quad.get_state(quaternion=True, stacked=True)
        quad_trajectory[:, current_idx] = quad_current_state


        #x_history.append(my_quad.get_state(quaternion=True))


    #my_viz.plot_position3D(quad_trajectory)

    #my_viz.plot_state_over_time(quad_trajectory)
    my_viz.plot_animated(quad_trajectory)

'''


'\n#os.environ["PYTHONBREAKPOINT"] = "0"\n\n\nreal_time_artists = None\n    \nif __name__ == "__main__":\n    my_quad = quad.Quadrotor3D()\n\n\n\n    simulation_steps = 20000\n    simulation_step_length = 0.1\n    x_history = list()\n    np.random.seed(0)\n    t = np.arange(0, simulation_steps*simulation_step_length, simulation_step_length)\n\n    quad_current_state = my_quad.get_state(quaternion=True, stacked=True)\n    quad_trajectory = np.zeros((13, simulation_steps))\n    #print(my_quad.get_state(quaternion=True))\n    #print(range(t))\n    for current_idx in tqdm(np.arange(1,simulation_steps)):\n        u = np.random.random((4,))\n        u = np.array([1,1,1,1])*10\n        #print(u)\n        my_quad.update(u, simulation_step_length)\n\n        #breakpoint()\n        #x_pred = np.zeros((13,1))\n        quad_current_state = my_quad.get_state(quaternion=True, stacked=True)\n        quad_trajectory[:, current_idx] = quad_current_state\n\n\n        #x_history.append(my_quad.get_state(

In [3]:
def quad_dynamics(rdrv_d=None):
    """
    Symbolic dynamics of the 3D quadrotor model. The state consists on: [p_xyz, a_wxyz, v_xyz, r_xyz]^T, where p
    stands for position, a for angle (in quaternion form), v for velocity and r for body rate. The input of the
    system is: [u_1, u_2, u_3, u_4], i.e. the activation of the four thrusters.

    :param rdrv_d: a 3x3 diagonal matrix containing the D matrix coefficients for a linear drag model as proposed
    by Faessler et al.

    :return: CasADi function that computes the analytical differential state dynamics of the quadrotor model.
    Inputs: 'x' state of quadrotor (6x1) and 'u' control input (2x1). Output: differential state vector 'x_dot'
    (6x1)
    """

    x_dot = cs.vertcat(p_dynamics(), q_dynamics(), v_dynamics(rdrv_d), w_dynamics())
    return cs.Function('x_dot', [x[:13], u], [x_dot], ['x', 'u'], ['x_dot'])

def p_dynamics():
    return v

def q_dynamics():
    return 1 / 2 * cs.mtimes(skew_symmetric(r), q)

def v_dynamics(rdrv_d):
    """
    :param rdrv_d: a 3x3 diagonal matrix containing the D matrix coefficients for a linear drag model as proposed
    by Faessler et al. None, if no linear compensation is to be used.
    """

    f_thrust = u * quad.max_thrust
    g = cs.vertcat(0.0, 0.0, 9.81)
    a_thrust = cs.vertcat(0.0, 0.0, f_thrust[0] + f_thrust[1] + f_thrust[2] + f_thrust[3]) / quad.mass

    v_dynamics = v_dot_q(a_thrust, q) - g

    if rdrv_d is not None:
        # Velocity in body frame:
        v_b = v_dot_q(v, quaternion_inverse(q))
        rdrv_drag = v_dot_q(cs.mtimes(rdrv_d, v_b), q)
        v_dynamics += rdrv_drag

    return v_dynamics

def w_dynamics():
    f_thrust = u * quad.max_thrust

    y_f = cs.MX(quad.y_f)
    x_f = cs.MX(quad.x_f)
    c_f = cs.MX(quad.z_l_tau)
    return cs.vertcat(
        (cs.mtimes(f_thrust.T, y_f) + (quad.J[1] - quad.J[2]) * r[1] * r[2]) / quad.J[0],
        (-cs.mtimes(f_thrust.T, x_f) + (quad.J[2] - quad.J[0]) * r[2] * r[0]) / quad.J[1],
        (cs.mtimes(f_thrust.T, c_f) + (quad.J[0] - quad.J[1]) * r[0] * r[1]) / quad.J[2])

def linearized_quad_dynamics():
    """
    Jacobian J matrix of the linearized dynamics specified in the function quad_dynamics. J[i, j] corresponds to
    the partial derivative of f_i(x) wrt x(j).

    :return: a CasADi symbolic function that calculates the 13 x 13 Jacobian matrix of the linearized simplified
    quadrotor dynamics
    """

    jac = cs.MX(state_dim, state_dim)

    # Position derivatives
    jac[0:3, 7:10] = cs.diag(cs.MX.ones(3))

    # Angle derivatives
    jac[3:7, 3:7] = skew_symmetric(r) / 2
    jac[3, 10:] = 1 / 2 * cs.horzcat(-q[1], -q[2], -q[3])
    jac[4, 10:] = 1 / 2 * cs.horzcat(q[0], -q[3], q[2])
    jac[5, 10:] = 1 / 2 * cs.horzcat(q[3], q[0], -q[1])
    jac[6, 10:] = 1 / 2 * cs.horzcat(-q[2], q[1], q[0])

    # Velocity derivatives
    a_u = (u[0] + u[1] + u[2] + u[3]) * quad.max_thrust / quad.mass
    jac[7, 3:7] = 2 * cs.horzcat(a_u * q[2], a_u * q[3], a_u * q[0], a_u * q[1])
    jac[8, 3:7] = 2 * cs.horzcat(-a_u * q[1], -a_u * q[0], a_u * q[3], a_u * q[2])
    jac[9, 3:7] = 2 * cs.horzcat(0, -2 * a_u * q[1], -2 * a_u * q[1], 0)

    # Rate derivatives
    jac[10, 10:] = (quad.J[1] - quad.J[2]) / quad.J[0] * cs.horzcat(0, r[2], r[1])
    jac[11, 10:] = (quad.J[2] - quad.J[0]) / quad.J[1] * cs.horzcat(r[2], 0, r[0])
    jac[12, 10:] = (quad.J[0] - quad.J[1]) / quad.J[2] * cs.horzcat(r[1], r[0], 0)

    return cs.Function('J', [x, u], [jac])

In [7]:
def acados_setup_model(nominal, model_name):
    """
    Builds an Acados symbolic models using CasADi expressions.
    :param model_name: name for the acados model. Must be different from previously used names or there may be
    problems loading the right model.
    :param nominal: CasADi symbolic nominal model of the quadrotor: f(self.x, self.u) = x_dot, dimensions 13x1.
    :return: Returns a total of three outputs, where m is the number of GP's in the GP ensemble, or 1 if no GP:
        - A dictionary of m AcadosModel of the GP-augmented quadrotor
        - A dictionary of m CasADi symbolic nominal dynamics equations with GP mean value augmentations (if with GP)
    :rtype: dict, dict, cs.MX
    """

    def fill_in_acados_model(x, u, p, dynamics, name):

        x_dot = cs.MX.sym('x_dot', dynamics.shape)
        f_impl = x_dot - dynamics

        # Dynamics model
        model = AcadosModel()
        model.f_expl_expr = dynamics
        model.f_impl_expr = f_impl
        model.x = x
        model.xdot = x_dot
        model.u = u
        model.p = p
        model.name = name

        return model

    acados_models = {}
    dynamics_equations = {}

    # Run GP inference if GP's available
    if self.gp_reg_ensemble is not None:

        # Feature vector are the elements of x and u determined by the selection matrix B_z. The trigger var is used
        # to select the gp-specific state estimate in the first optimization node, and the regular integrated state
        # in the rest. The computing of the features z is done within the GPEnsemble.
        gp_x = self.gp_x * self.trigger_var + self.x * (1 - self.trigger_var)
        #  Transform velocity to body frame
        v_b = v_dot_q(gp_x[7:10], quaternion_inverse(gp_x[3:7]))
        gp_x = cs.vertcat(gp_x[:7], v_b, gp_x[10:])
        gp_u = self.u

        gp_dims = self.gp_reg_ensemble.dim_idx

        # Get number of models in GP
        for i in range(self.gp_reg_ensemble.n_models):
            # Evaluate cluster of the GP ensemble
            cluster_id = {k: [v] for k, v in zip(gp_dims, i * np.ones_like(gp_dims, dtype=int))}
            outs = self.gp_reg_ensemble.predict(gp_x, gp_u, return_cov=False, gp_idx=cluster_id, return_z=False)

            # Unpack prediction outputs. Transform back to world reference frame
            outs = self.add_missing_states(outs)
            gp_means = v_dot_q(outs["pred"], gp_x[3:7])
            gp_means = self.remove_extra_states(gp_means)

            # Add GP mean prediction
            dynamics_equations[i] = nominal + cs.mtimes(self.B_x, gp_means)

            x_ = self.x
            dynamics_ = dynamics_equations[i]

            # Add again the gp augmented dynamics for the GP state
            dynamics_ = cs.vertcat(dynamics_)
            dynamics_equations[i] = cs.vertcat(dynamics_equations[i])

            i_name = model_name + "_domain_" + str(i)

            params = cs.vertcat(self.gp_x, self.trigger_var)
            acados_models[i] = fill_in_acados_model(x=x_, u=self.u, p=params, dynamics=dynamics_, name=i_name)

    else:

        # No available GP so return nominal dynamics
        dynamics_equations[0] = nominal

        x_ = self.x
        dynamics_ = nominal

        acados_models[0] = fill_in_acados_model(x=x_, u=self.u, p=[], dynamics=dynamics_, name=model_name)

    return acados_models, dynamics_equations

In [10]:


# Weighted squared error loss function q = (p_xyz, a_xyz, v_xyz, r_xyz), r = (u1, u2, u3, u4)
q_cost = np.array([10, 10, 10, 0.1, 0.1, 0.1, 0.05, 0.05, 0.05, 0.05, 0.05, 0.05])
r_cost = np.array([0.1, 0.1, 0.1, 0.1])

# Time horizon for MPC optimalization
t_horizon=1
n_nodes=20
model_name="quad_3d_acados_mpc"

rdrv_d_mat = None # a 3x3 diagonal matrix containing the D matrix coefficients for a linear drag model as proposed by Faessler et al.

T = t_horizon  # Time horizon
N = n_nodes  # number of control nodes within horizon

quad = Quadrotor3D()

max_u = quad.max_input_value
min_u = quad.min_input_value

# Declare model variables
p = cs.MX.sym('p', 3)  # position
q = cs.MX.sym('a', 4)  # angle quaternion (wxyz)
v = cs.MX.sym('v', 3)  # velocity
r = cs.MX.sym('r', 3)  # angle rate

# Full state vector (13-dimensional)
x = cs.vertcat(p, q, v, r)
state_dim = 13

# Control input vector
u1 = cs.MX.sym('u1')
u2 = cs.MX.sym('u2')
u3 = cs.MX.sym('u3')
u4 = cs.MX.sym('u4')
u = cs.vertcat(u1, u2, u3, u4)

# Nominal model equations symbolic function (no GP)
quad_xdot_nominal = quad_dynamics(rdrv_d_mat)

# Linearized model dynamics symbolic function
quad_xdot_jac = linearized_quad_dynamics()

# Initialize objective function, 0 target state and integration equations
L = None
target = None

# Check if GP ensemble has an homogeneous feature space (if actual Ensemble)
gp_reg_ensemble = None

# Declare model variables for GP prediction (only used in real quadrotor flight with EKF estimator).
# Will be used as initial state for GP prediction as Acados parameters.
gp_p = cs.MX.sym('gp_p', 3)
gp_q = cs.MX.sym('gp_a', 4)
gp_v = cs.MX.sym('gp_v', 3)
gp_r = cs.MX.sym('gp_r', 3)
gp_x = cs.vertcat(gp_p, gp_q, gp_v, gp_r)

# The trigger variable is used to tell ACADOS to use the additional GP state estimate in the first optimization
# node and the regular integrated state in the rest
trigger_var = cs.MX.sym('trigger', 1)

# Build full model. Will have 13 variables. dyn_x contains the symbolic variable that
# should be used to evaluate the dynamics function. It corresponds to x if there are no GP's, or
# x_with_gp otherwise
acados_models, nominal_with_gp = acados_setup_model(
    quad_xdot_nominal(x=x, u=u)['x_dot'], model_name)

# Convert dynamics variables to functions of the state and input vectors
quad_xdot = {}
for dyn_model_idx in nominal_with_gp.keys():
    dyn = nominal_with_gp[dyn_model_idx]
    quad_xdot[dyn_model_idx] = cs.Function('x_dot', [x, u], [dyn], ['x', 'u'], ['x_dot'])

# ### Setup and compile Acados OCP solvers ### #
acados_ocp_solver = {}

# Check if GP's have been loaded
with_gp = gp_reg_ensemble is not None

# Add one more weight to the rotation (use quaternion norm weighting in acados)
q_diagonal = np.concatenate((q_cost[:3], np.mean(q_cost[3:6])[np.newaxis], q_cost[3:]))
if q_mask is not None:
    q_mask = np.concatenate((q_mask[:3], np.zeros(1), q_mask[3:]))
    q_diagonal *= q_mask

# Ensure current working directory is current folder
os.chdir(os.path.dirname(os.path.realpath(__file__)))
acados_models_dir = '../../acados_models'
safe_mkdir_recursive(os.path.join(os.getcwd(), acados_models_dir))

for key, key_model in zip(acados_models.keys(), acados_models.values()):
    nx = key_model.x.size()[0]
    nu = key_model.u.size()[0]
    ny = nx + nu
    n_param = key_model.p.size()[0] if isinstance(key_model.p, cs.MX) else 0

    acados_source_path = os.environ['ACADOS_SOURCE_DIR']
    sys.path.insert(0, '../common')

    # Create OCP object to formulate the optimization
    ocp = AcadosOcp()
    ocp.acados_include_path = acados_source_path + '/include'
    ocp.acados_lib_path = acados_source_path + '/lib'
    ocp.model = key_model
    ocp.dims.N = N
    ocp.solver_options.tf = t_horizon

    # Initialize parameters
    ocp.dims.np = n_param
    ocp.parameter_values = np.zeros(n_param)

    ocp.cost.cost_type = 'LINEAR_LS'
    ocp.cost.cost_type_e = 'LINEAR_LS'

    ocp.cost.W = np.diag(np.concatenate((q_diagonal, r_cost)))
    ocp.cost.W_e = np.diag(q_diagonal)
    terminal_cost = 0 if solver_options is None or not solver_options["terminal_cost"] else 1
    ocp.cost.W_e *= terminal_cost

    ocp.cost.Vx = np.zeros((ny, nx))
    ocp.cost.Vx[:nx, :nx] = np.eye(nx)
    ocp.cost.Vu = np.zeros((ny, nu))
    ocp.cost.Vu[-4:, -4:] = np.eye(nu)

    ocp.cost.Vx_e = np.eye(nx)

    # Initial reference trajectory (will be overwritten)
    x_ref = np.zeros(nx)
    ocp.cost.yref = np.concatenate((x_ref, np.array([0.0, 0.0, 0.0, 0.0])))
    ocp.cost.yref_e = x_ref

    # Initial state (will be overwritten)
    ocp.constraints.x0 = x_ref

    # Set constraints
    ocp.constraints.lbu = np.array([min_u] * 4)
    ocp.constraints.ubu = np.array([max_u] * 4)
    ocp.constraints.idxbu = np.array([0, 1, 2, 3])

    # Solver options
    ocp.solver_options.qp_solver = 'FULL_CONDENSING_HPIPM'
    ocp.solver_options.hessian_approx = 'GAUSS_NEWTON'
    ocp.solver_options.integrator_type = 'ERK'
    ocp.solver_options.print_level = 0
    ocp.solver_options.nlp_solver_type = 'SQP_RTI' if solver_options is None else solver_options["solver_type"]

    # Compile acados OCP solver if necessary
    json_file = os.path.join(acados_models_dir, key_model.name + '_acados_ocp.json')
    acados_ocp_solver[key] = AcadosOcpSolver(ocp, json_file=json_file)

quad_opt = Quad3DOptimizer(quad.Quadrotor3D())

TypeError: acados_setup_model() missing 1 required positional argument: 'model_name'

In [ ]:
my_viz.plot_position3D(quad_trajectory)

In [ ]:
def animate(i, x_trajectory, fig, world_rad):


    ax = fig.add_subplot(1, 2, 1, projection='3d')

    ax.clear()

    # +1 because of the trajectory 
    ax.plot3D(x_trajectory[0,:i+1], x_trajectory[1,:i+1], x_trajectory[2,:i+1], 'gray') 
    # current location
    ax.scatter(x_trajectory[0,i], x_trajectory[1,i], x_trajectory[2,i])

    ax.set_xlim([-world_rad, world_rad])
    ax.set_ylim([-world_rad, world_rad])
    ax.set_zlim([-world_rad, world_rad])

In [ ]:
fig = plt.figure()
world_rad = 5
anim = lambda i : animate(i, quad_trajectory, fig, world_rad)
line_ani = FuncAnimation(fig, anim, interval=100,   
                                   frames=quad_trajectory.shape[0])
plt.show()